In [1]:
#import libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np

In [2]:
#access the webdriver
driver = webdriver.Chrome('/Users/meenu/Downloads/chromedriver')
driver=webdriver.Chrome()


C:\Users\meenu\AppData\Local\Temp\ipykernel_10236\4000343727.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/meenu/Downloads/chromedriver')


In [3]:
# create empty lists to store the data
product = []
company_name = []
features = []
price = []

In [4]:
# create an empty DataFrame to store the scraped data
df = pd.DataFrame()

# loop through the pages
for page in range(1, 3): # modify the range based on the number of pages you want to scrape
    # create the url
    url = f'https://tyresnmore.com/all/car-tyres/toyota/innova.html?p={page}'

    # navigate to the url
    driver.get(url)
    product = [i.text for i in driver.find_elements(By.XPATH, '//span[@class="product-list-name"]')]
    
    company_name = [s.split()[0] for s in product]
    df1 = pd.DataFrame({ 'Company': company_name,'Product': product})
    
    features = [fe.text.split('\n\n') for fe in driver.find_elements(By.XPATH, '//div[@class="clsProductFeature"]')]

    # create a set of unique features
    unique_features = set([feature for product_features in features for feature in product_features])

    # create an empty dictionary to store the encoded features
    encoded_features = {feature: [] for feature in unique_features}

    # encode the features
    for product_features in features:
        for feature in unique_features:
            if feature in product_features:
                encoded_features[feature].append(1)
            else:
                encoded_features[feature].append(0)
                
    #created another dataframe that contains product names and their corresponding encoded features
    encoded_df = pd.DataFrame(encoded_features)
    encoded_df.insert(0, "Product", product)

    # merge this dataframe with the existing dataframe
    merged_df = pd.merge(df1, encoded_df, on="Product")
    
    # Find all prices
    prices = driver.find_elements(By.XPATH, '//span[@class="amt_price price-final_price tax weee"]')

    new_price_list = []
    for i in prices:
        text = i.text
        if 'Special Price' in text:
            new_price_list.append('Special Price')
            new_price_list.append(text.split("\n")[1])
        else:
            new_price_list.append(text)
        
    # new_list contain the original price and special price of each product. 
    # So we delete the original price if special price exist for any of the products.

    i = 0
    while i < len(new_price_list):
        if new_price_list[i] == "Special Price":
            del new_price_list[i-1:i+1]
        else:
            i += 1
    merged_df['Price'] = new_price_list
    
    # append the merged_df to the main df
    df = pd.concat([df, merged_df], ignore_index=True)
    
#replace all the null values with 0
df = df.fillna(0)
#convert BRIDGESTONE TO Bridgestone
df.loc[df['Company'] == 'BRIDGESTONE', 'Company'] = 'Bridgestone'

# print the final dataframe
print(df)



        Company                                            Product  \
0          CEAT               CEAT 205/65 R15 MILAZE X3 94S TL TWI   
1      Goodyear  Goodyear 205/65 R15 Assurance Duraplus 2 99H T...   
2      Goodyear  Goodyear 265/65 R17 Wrangler AT/ST 112S Tubele...   
3      YOKOHAMA  YOKOHAMA 205/65 R16 EARTH-1 95H TUBELESS CAR TYRE   
4      YOKOHAMA  YOKOHAMA 205/65 R15 EARTH-1 94H TUBELESS CAR TYRE   
5      YOKOHAMA  YOKOHAMA 265/65 R17 GEOLANDAR G015 112H TUBELE...   
6         Kelly       Kelly 205/65 R15 VFM 6 94H Tubeless Car Tyre   
7     ULTRAMILE  ULTRAMILE 205/65 R15 UM551 BSW 94H TUBELESS CA...   
8        Apollo   Apollo 205/65 R15 ALNAC 4G 94H Tubeless Car Tyre   
9        Apollo  Apollo 205/65 R15 AMAZER 4G LIFE 94T Tubeless ...   
10          MRF                        MRF 205/65 R15 ZTX-S 94H TL   
11      PIRELLI    PIRELLI 205/65 R15 P6 CIN 94V TUBELESS CAR TYRE   
12  Bridgestone                 BRIDGESTONE 205/65 R15 B390 94S TL   
13  Bridgestone   Br

In [5]:
#convert it into csv file
df.to_csv('tyresnmore.csv', index=False)